In [1]:
from codelin.models.const_tree import C_Tree
from codelin.encs.constituent import *
from codelin.utils.constants import *
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.const_label import C_Label
from nltk.tree import Tree

import pandas as pd
import os
import copy

def pt(t, d=True):
    '''
    Print tree if d is True
    '''
    if d:
        Tree.fromstring(str(t)).pretty_print()

def bin_print(ts):
    '''
    Print binary tree
    '''
    Tree.fromstring(str(C_Tree.to_binary_right(C_Tree.from_string(ts)))).pretty_print()

tsa = "(NTA (PA a) (NTB (NTC (NTD (PB b) (NTE (PC c) (NTF (NTG (PD d) (PE e)) (PF f)))) (PG g)) (PH h)))"
tsb = "(NTA (PA a) (NTB (NTC (PB b) (NTF (NTD (PC c) (NTE (PD d) (PE e))) (PF f))) (NTG (NTH (PG g) (NTI (PH h) (NTJ (PI i) (PJ j)))) (PK k))))"
tsc = "(NTA (NTU (NTV (PA11 a11) (PA12 a12)) (NTW (PA21 a21) (PA22 a22))) (NTB (NTC (PB b) (NTF (NTD (PC c) (NTE (PD d) (PE e))) (PF f))) (NTG (NTH (PG g) (NTI (PH h) (NTJ (PI i) (PJ j)))) (PK k))))"
tsd = "(NTA (NTU (NTV (NTP (S s) (T t)) (PA12 a12)) (NTW (PA21 a21) (NTQ (X x) (Y y)))) (NTB (NTC (PB b) (NTF (NTD (PC c) (NTE (PD d) (PE e))) (PF f))) (NTG (NTH (PG g) (NTI (PH h) (NTJ (PI i) (PJ j)))) (PK k))))"

ts1 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
ts2 = "(SINV (S (ADVP (RB Once) (RB again)) (-LRB- -LCB-) (NP (DT the) (NNS specialists)) (-RRB- -RCB-) (VP (VBD were) (RB not) (ADJP (JJ able) (S (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Christopher) (NNP Pedersen)) (, ,) (NP (NP (JJ senior) (NN vice) (NN president)) (PP (IN at) (NP (NNP Twenty-First) (NNP Securities) (NNP Corp))))) (. .))"
ts3 = "(S (NP (NP (NNP Seven) (NNP Big) (NNP Board) (NNS stocks)) (: --) (NP (NP (NNP UAL)) (, ,) (NP (NNP AMR)) (, ,) (NP (NNP BankAmerica)) (, ,) (NP (NNP Walt) (NNP Disney)) (, ,) (NP (NNP Capital) (NNP Cities\/ABC)) (, ,) (NP (NNP Philip) (NNP Morris)) (CC and) (NP (NNP Pacific) (NNP Telesis) (NNP Group))) (: --)) (VP (VP (VBD stopped) (S (VP (NN trading)))) (CC and) (VP (ADVP (RB never)) (VBD resumed))) (. .))"
ts4 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
ts5 = "(FRAG (SBAR (IN As) (IN in) (: :) (`` ``) (SQ (NP (PRP You)) (VP (VBD went) (NP (VBG ballooning))) (. ?) (. ?) (. !) (. !))))"
ts6 = "(NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN \*)))"
ts7 = "(S (INTJ (RB No)) (, ,) (NP (PRP it)) (VP (VBD was) (RB n't) (NP (NNP Black) (NNP Monday))) (. .))"
ts8 = "(S (NP (DT The) (JJ finger-pointing)) (VP (VBZ has) (ADVP (RB already)) (VP (VBN begun))) (. .))"
ts9 = "(S (`` ``) (NP (DT The) (NN equity) (NN market)) (VP (VBD was) (ADJP (JJ illiquid))) (. .))"
ts10 = "(S (PP (IN In) (NP (NP (DT an) (NNP Oct.) (CD 19) (NN review)) (PP (IN of) (NP (`` ``) (NP (DT The) (NN Misanthrope)) ('' '') (PP (IN at) (NP (NP (NNP Chicago) (POS 's)) (NNP Goodman) (NNP Theatre))))) (PRN (-LRB- -LRB-) (`` ``) (S (NP (JJ Revitalized) (NNS Classics)) (VP (VBP Take) (NP (DT the) (NNP Stage)) (PP (IN in) (NP (NNP Windy) (NNP City))))) (, ,) ('' '') (NP (NNP Leisure) (CC &) (NNP Arts)) (-RRB- -RRB-)))) (, ,) (NP (NP (NP (DT the) (NN role)) (PP (IN of) (NP (NNP Celimene)))) (, ,) (VP (VBN played) (PP (IN by) (NP (NNP Kim) (NNP Cattrall)))) (, ,)) (VP (VBD was) (VP (ADVP (RB mistakenly)) (VBN attributed) (PP (TO to) (NP (NNP Christina) (NNP Haag))))) (. .))"

/home/poli/.conda/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Encoding playground

In [36]:
# GAP OPERATOR
def insert_children_on_empty_child(tree, child):
    if tree.children[0].label == "EMPTY":
        tree.children[0] = child
    else:
        tree.children[1] = child
    return tree
def generate_leaf_tree(postag, word):
    return C_Tree(label=postag, children=C_Tree(label=word))
def generate_empty_tree():
    return C_Tree(label="EMPTY", children=C_Tree(label="EMPTY"))

def operator_1(gap_stack, stack, word, last_common):
    # > Create new sub-tree with label = last_common, left_child=Tree(postag, word), right_child=None and push to stack
    new_tree = C_Tree(label=last_common, children=[generate_leaf_tree(postag="P", word=word), generate_empty_tree()])
    stack.append(new_tree)
    
    return gap_stack, stack

def operator_open_gap(gap_stack, stack, word, last_common):
    # > Create new sub-tree with label = last_common, left_child=Tree(postag, word), right_child=None and push to stack
    new_tree = C_Tree(label=last_common, children=[generate_leaf_tree(postag="P", word=word),  generate_empty_tree()])
    stack.append(new_tree)

    # > Push new sub-tree to open_gaps_stack
    gap_stack.append(new_tree)
    return gap_stack, stack


def operator_n1(gap_stack, stack, word, last_common):
    # Subir 1 e insertar?
    # > Create new sub-tree with label = last_common, left_child=popped_tree, right_child=None and push to stack    
    stack_top = stack.pop()

    # check if the gap that we are closing is still in the gap stack
    # if stack_top in gap_stack:
    #     gap_stack.remove(stack_top)

    new_children = generate_leaf_tree(postag="P", word=word)
    insert_children_on_empty_child(stack_top, new_children)
    
    # > Pop stack and insert a Tree(postag, word) in the empty child of the popped tree
    new_tree = C_Tree(label=last_common, children=[stack_top,  generate_empty_tree()])
    stack.append(new_tree)

    return gap_stack, stack

def is_subtree(t1, t2):
    t1t = t1.get_terminals()
    t2t = t2.get_terminals()
    return all (item in t1t for item in t2t) or all (item in t2t for item in t1t)

def check_tree_in_stack(tree, stack):
    for element in stack:
        if is_subtree(element, tree):
            return True
    return False

def operator_close_gap(gap_stack, stack, word, last_common):
    # > Pop stack and insert a Tree(postag, word) in the empty child of the popped tree
    stack_top = stack.pop()
    new_children = generate_leaf_tree(postag="P", word=word)
    insert_children_on_empty_child(stack_top, new_children)
    
    # If we have more than open gaps, we need to find the last open gap that is still in the stack
    last_open_gap_position_index = 0
    if len(gap_stack) >= 1:
        last_open_gap = gap_stack.pop()    
        while not "EMPTY" in last_open_gap.children[1].label and not check_tree_in_stack(last_open_gap, stack) and len(gap_stack) > 0:
            last_open_gap = gap_stack.pop()
        
        last_open_gap_position_index = 0
        for i in range(len(stack)):
            if is_subtree(stack[i], last_open_gap):
                last_open_gap_position_index = i
                break
    
    # > Pop stack until we reach the last open gap
    #       => this loop will run at least once
    #       => this loop will merge all elements in the stack from the top until the 'last open gap'
    for i in range(len(stack), last_open_gap_position_index, -1):
        new_top = stack[-1]
        if check_tree_in_stack(new_top, gap_stack):
            break
        else:
            new_top = stack.pop()
            insert_children_on_empty_child(new_top, stack_top)
            stack_top = new_top



    # > Create new sub-tree with label = last_common, left_child=popped_tree, right_child=None and push to stack
    new_tree = C_Tree(label=last_common, children=[stack_top, generate_empty_tree()])
    stack.append(new_tree)

    return gap_stack, stack

ts = ts4
debug = False
encoder = C_NaiveRelativeEncoding(separator="_",
                                    unary_joiner="++",
                                    reverse=False,
                                    binary=True,
                                    binary_direction="R",
                                    binary_marker="*")
tencoder = C_Tetratag(separator="_",
                        unary_joiner="++",
                        reverse=False,
                        binary_direction="R",
                        binary_marker="*")

# ptb_path = "/home/poli/Documents/Treebanks/PENN_TREEBANK/train.trees"
ptb_path="/home/poli/Treebanks/const/PENN_TREEBANK/dev.trees"
ptb_file = open(ptb_path, "r")


# good examples: 2,4,6,9
# bad example: 32

idx     = 9
use_n1  = True
for line in ptb_file.readlines()[idx:idx+1]:

    tree = C_Tree.from_string(line.rstrip())
    lc_tree = encoder.encode(copy.deepcopy(tree))
    lc_tree_tetratag = tencoder.encode(copy.deepcopy(tree))

    for i in range(len(lc_tree.labels)):
        if lc_tree.labels[i].n_commons == -1:
            lc_tree.labels[i-1].n_commons = lc_tree.labels[i-1].n_commons-1


    new_labels = []
    for label, tetra_label in zip(lc_tree.labels, lc_tree_tetratag.labels):
        new_nc = ""
        if lc_tree.labels.index(label)==0:
            new_nc = "G"
        elif lc_tree.labels.index(label)==len(lc_tree.labels)-1:
            new_nc = "-G"
        
        elif label.n_commons == 1:
            new_nc = "1"
        elif label.n_commons == -1:
            new_nc = "-1"
        elif label.n_commons < 0:
            new_nc = "-G"
        else:
            new_nc = "G"
        
        lc = label.last_common if lc_tree.labels.index(label)!=len(lc_tree.labels)-1 else "$$"
        new_labels.append((new_nc, lc, label.n_commons, tetra_label.n_commons))

    gap_stack = []
    stack     = []
    print("Original:")
    tree = C_Tree.from_string(line.rstrip())
    tree = tree.collapse_unary()
    tree = C_Tree.to_binary_right(tree)
    pt(tree)
    print("\n\n")

    for label, word in zip(new_labels, lc_tree.words):
        print(f"{word:<15}  {label[1]:<7} {label[0]:<5} {label[2]:<5} {label[3]:<5}")
        if label[0] == "1":
            gap_stack, stack = operator_1(gap_stack, stack, word, label[1])
        elif label[0] == "-1":
            # gap_stack, stack = operator_close_gap(gap_stack, stack, word, label[1])
            gap_stack, stack = operator_n1(gap_stack, stack, word, label[1])
        elif label[0] == "G":
            gap_stack, stack = operator_open_gap(gap_stack, stack, word, label[1])
        elif label[0] == "-G":
            gap_stack, stack = operator_close_gap(gap_stack, stack, word, label[1])
        else:
            raise ValueError("Unknown label: {}".format(label))
        
        # print("#######STACK#######\n")
        # for element in stack:    
        #     Tree.fromstring(str(element)).pretty_print()
        #     print("-------------------")
        # print("#######GAP_STACK#######\n")
        # for element in gap_stack:
        #     Tree.fromstring(str(element)).pretty_print()
        #     print("-------------------")

    print("==>",len(stack))
    if len(stack)>1:
        print("merging stack")
        while len(stack)>1:
            new_top = stack.pop()
            stack[-1] = insert_children_on_empty_child(stack[-1], new_top)
            

    result_tree = stack[0].children[0]
    pt(result_tree)
    # result_tree_restored = C_Tree.restore_from_binary(result_tree)

    # esto falla por las unary chains !!
    # print("Decoded:")
    # bin_print(str(result_tree))

Original:
                                 SINV                                                                                                                           
  ________________________________|______________________________________________                                                                                
 |                                                                             SINV*                                                                            
 |           ____________________________________________________________________|_____                                                                          
 |          |                                                                        SINV*                                                                      
 |          |                                                                     _____|____________                                                             
 |          S        

In [3]:
raise ValueError("Done")

ValueError: Done

## Labels equivalence

In [33]:
ptb_path = "~/Treebanks/const/PENN_TREEBANK/"
ptb_path = os.path.expanduser(ptb_path)
print("Reading PTB from",ptb_path)    
df = pd.DataFrame(columns=["Word", "Relative", "Tetratag"])
with open(os.path.join(ptb_path,"test.trees")) as f:
    ptb = f.read().splitlines()

for tree_string in ptb:
    tree = C_Tree.from_string(tree_string)
    rel_encoder = C_NaiveRelativeEncoding(separator="_", unary_joiner="+", reverse=False, binary=True, binary_marker="[*]", binary_direction="R")
    tet_encoder = C_Tetratag(separator="_", unary_joiner="+", reverse=False, binary_marker="[*]", binary_direction="R")

    t_lin_tree = tet_encoder.encode(copy.deepcopy(tree))
    r_lin_tree = rel_encoder.encode(copy.deepcopy(tree))

    df_temp = pd.DataFrame(columns=["Word", "Relative", "Tetratag"])
    
    df_temp["Word"] = r_lin_tree.words
    df_temp["Relative"] = [str(l.n_commons) for l in r_lin_tree.labels]
    df_temp["Tetratag"] = [l.n_commons for l in t_lin_tree.labels]
    
    df = pd.concat([df, df_temp], axis=0)

    r_decoded_tree = rel_encoder.decode(r_lin_tree)
    r_decoded_tree = r_decoded_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
    
    t_decoded_tree = tet_encoder.decode(t_lin_tree)
    t_decoded_tree = t_decoded_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
        
    # if not (str(r_decoded_tree)==tree_string):
    #     print("Decoded tree is different from original")
    #     print("Original:",tree_string)
    #     print("Decoded:",r_decoded_tree)

Reading PTB from /home/poli/Treebanks/const/PENN_TREEBANK/


In [34]:
# print relative values for each of the unique values of tetratag
tetra_values = df["Tetratag"].unique()
tetra_values.sort()
for tetratag in tetra_values:
    # ignore 'close' labels
    if len(tetratag)==1:
        continue
    
    # sort
    rel_values = df[df["Tetratag"]==tetratag]["Relative"].unique()
    rel_values = [int(v) for v in rel_values]
    rel_values.sort()
    print("Tetratag:",tetratag,"\n\t==> Relative:",rel_values)

Tetratag: lL 
	==> Relative: [-31, -30, -27, -26, -25, -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]
Tetratag: lR 
	==> Relative: [-21, -20, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]
Tetratag: rL 
	==> Relative: [1]
Tetratag: rR 
	==> Relative: [1, 2, 3, 4, 5]
